In [7]:
import imp
import Data
import utils
import Predictors, Road_Graph
imp.reload(Data)
imp.reload(utils)
imp.reload(Predictors)
imp.reload(Road_Graph)

from Road_Graph import *
from Predictors import *
from Data import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch
from torch import nn, autograd
import torch.nn.functional as F

In this part, we focus on a simple road map (rep as a graph below), and generate random path on it.
<img src="../img/naive_road.png" alt="Drawing" style="width: 400px;"/>

# build the road graph as above


In [8]:
graph_dict = {}
graph_dict = graph_dict.fromkeys(range(24))
edges = [(i, i+1) for i in range(5)] \
        + [(i+6, i+7) for i in range(5)]\
                        +[(i+12, i+13) for i in range(5)]\
                     + [(i+18, i+19) for i in range(5)]\
                     + [(0,6),(6,12), (12,18)]\
                     + [(pair[0]+1, pair[1]+1) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+2, pair[1]+2) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+3, pair[1]+3) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+4, pair[1]+4) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+5, pair[1]+5) for pair in [(0,6),(6,12), (12,18)]]
for i,j in edges:
    if not graph_dict[i]:
        graph_dict[i] = [j]
    else:
        graph_dict[i].append(j)
    if not graph_dict[j]:
        graph_dict[j] = [i]
    else:
        graph_dict[j].append(i)

graph = Road_Graph(graph_dict)

In [9]:
print('nodes are {}, \nedges are\n {}'.format(graph.nodes, graph.edges))
print('neighbors of node 1 is {}'.format(graph.neighbors(1)))

nodes are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23], 
edges are
 [[ 0  1]
 [ 0  6]
 [ 1  0]
 [ 1  2]
 [ 1  7]
 [ 2  1]
 [ 2  3]
 [ 2  8]
 [ 3  2]
 [ 3  4]
 [ 3  9]
 [ 4  3]
 [ 4  5]
 [ 4 10]
 [ 5  4]
 [ 5 11]
 [ 6  7]
 [ 6  0]
 [ 6 12]
 [ 7  6]
 [ 7  8]
 [ 7  1]
 [ 7 13]
 [ 8  7]
 [ 8  9]
 [ 8  2]
 [ 8 14]
 [ 9  8]
 [ 9 10]
 [ 9  3]
 [ 9 15]
 [10  9]
 [10 11]
 [10  4]
 [10 16]
 [11 10]
 [11  5]
 [11 17]
 [12 13]
 [12  6]
 [12 18]
 [13 12]
 [13 14]
 [13  7]
 [13 19]
 [14 13]
 [14 15]
 [14  8]
 [14 20]
 [15 14]
 [15 16]
 [15  9]
 [15 21]
 [16 15]
 [16 17]
 [16 10]
 [16 22]
 [17 16]
 [17 11]
 [17 23]
 [18 19]
 [18 12]
 [19 18]
 [19 20]
 [19 13]
 [20 19]
 [20 21]
 [20 14]
 [21 20]
 [21 22]
 [21 15]
 [22 21]
 [22 23]
 [22 16]
 [23 22]
 [23 17]]
neighbors of node 1 is [0 2 7]


In [10]:
data = random_walk_data(graph, 10000,go_back = True)
display(data)

[array([ 1,  7, 13, 19, 20, 19, 13,  7]),
 array([11, 17]),
 array([11, 10]),
 array([ 5, 11,  5,  4, 10,  4,  3,  4]),
 array([ 6,  0,  6,  0,  1,  7, 13, 14, 13, 19, 20, 21, 15]),
 array([17, 23, 22, 16, 15, 21, 15, 16, 17, 23, 22, 21, 22, 21, 22, 23, 22,
        21, 22, 23, 22, 21, 20, 21, 20, 21, 15, 14]),
 array([20, 19, 18, 12]),
 array([18, 19, 13, 19]),
 array([18, 12, 18, 12, 18, 12, 13, 12, 13]),
 array([12, 13, 14]),
 array([ 7,  8, 14, 13,  7,  6, 12]),
 array([20, 19, 13,  7, 13,  7,  1,  0,  1,  0]),
 array([20, 21, 15, 21, 20, 21, 20]),
 array([4, 5, 4, 3]),
 array([22, 23, 22, 16]),
 array([3, 2, 1, 2]),
 array([20, 21, 15]),
 array([13, 19, 20, 21, 15, 16, 22, 21, 22, 23]),
 array([ 3,  9,  8,  9,  8, 14, 15, 21, 22, 21, 15, 14,  8,  7,  6]),
 array([23, 22, 16, 10, 16, 10,  4, 10,  9,  8,  2,  1,  7,  6,  0,  1,  0,
         1,  2,  1,  0,  6,  0,  1,  2,  1,  2]),
 array([1, 2, 8, 9]),
 array([15, 16, 22, 21, 20, 21, 22, 16, 15, 14, 13, 19, 20, 19]),
 array([22, 16, 

In [11]:
mk_pd = Markov_Predictor(graph) # construct a markov predictor on the road network 
mk_pd.train(data) # train the markov predictor
nxt,prob = mk_pd.predict([0,1,2]) # given a path [0,1,2], predict the probability for next road
print(nxt, prob)
mk_pd.eval(random_walk_data(graph, 1000,go_back = True)) # evaluate the markov model

[1 3 8] [ 0.33139241  0.33037975  0.33822785]
Processed 1000 road!
Processed 2000 road!
Processed 3000 road!
Processed 4000 road!
Processed 5000 road!
Processed 6000 road!
Processed 7000 road!
Processed 8000 road!
Processed 9000 road!


1.1672978696093785